In [1]:
import gzip
from io import StringIO
from pathlib import Path

import pandas as pd
import requests
import xmltodict

In [2]:
# Set base urls

data_source_url = "http://noaa-ghcn-pds.s3.amazonaws.com"
dest_dir = Path("C:/Users/dhruv/noaa-ghcnd")
stations_url = "http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt"

In [3]:
def download_stations(stations_url):
    station_response = requests.get(stations_url)
    df = pd.read_fwf(StringIO(station_response.content.decode()),
                     header=None,
                     colspecs=[
                         (0, 11),
                         (12, 20),
                         (21, 30),
                         (31, 37),
                         (38, 40),
                         (41, 71),
                         (72, 75),
                         (76, 79),
                         (80, 85)
                     ]
                     )
    df.columns = [
        'station_id',
        'latitude',
        'longitude',
        'elevation',
        'state',
        'name',
        'gsn_flag',
        'hcn_crn_flag',
        'wmo_id'
    ]
    return df

In [4]:
stations = download_stations(stations_url)
stations.to_parquet(dest_dir / "stations.parquet")
stations.head(10)

,station_id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0
5,AEM00041218,24.2620,55.6090,264.9,NaN,AL AIN INTL,NaN,NaN,41218.0
6,AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
7,AFM00040938,34.2100,62.2280,977.2,NaN,HERAT,NaN,NaN,40938.0
8,AFM00040948,34.5660,69.2120,1791.3,NaN,KABUL INTL,NaN,NaN,40948.0
9,AFM00040990,31.5000,65.8500,1010.0,NaN,KANDAHAR AIRPORT,NaN,NaN,40990.0


In [5]:
def get_available_files(url=data_source_url):
    manifest_response = requests.get(data_source_url)
    manifest = xmltodict.parse(manifest_response.content)

    # filter manifest for files by year
    wanted = []
    for file in manifest['ListBucketResult']['Contents']:
        if "by_station" not in file['Key']:
            wanted.append(file['Key'])
    return wanted

In [6]:
# define download function
def download(key):
    url = f"{data_source_url}/{key}"
    response = requests.get(url)

    filename = key.split("/")[-1]
    csv_content = gzip.decompress(response.content)
    memfile = StringIO(csv_content.decode())
    df = pd.read_csv(memfile, sep=",", header=None)
    return df

In [7]:
files = get_available_files()
raw = download(files[-1])
raw.head(10)

,0,1,2,3,4,5,6,7
0,AE000041196,20220101,TAVG,204,H,NaN,S,NaN
1,AEM00041194,20220101,TAVG,211,H,NaN,S,NaN
2,AEM00041217,20220101,TAVG,209,H,NaN,S,NaN
3,AEM00041218,20220101,TAVG,207,H,NaN,S,NaN
4,AG000060390,20220101,TAVG,121,H,NaN,S,NaN
5,AG000060590,20220101,TAVG,151,H,NaN,S,NaN
6,AG000060611,20220101,TAVG,111,H,NaN,S,NaN
7,AGE00147708,20220101,TMIN,73,NaN,NaN,S,NaN
8,AGE00147708,20220101,PRCP,0,NaN,NaN,S,NaN
9,AGE00147708,20220101,TAVG,133,H,NaN,S,NaN


In [8]:
df = raw.copy()
df.columns = [
    'station_id',
    'date',
    'element',
    'value',
    'm_flag',
    'q_flag',
    's_flag',
    'obs_time',
]

df['station_id'] = df['station_id'].astype(pd.CategoricalDtype(categories=None, ordered=False))
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['element'] = df['element'].astype(pd.CategoricalDtype(categories=None, ordered=False))
df

## create a json which holds hardcoded metadata

,station_id,date,element,value,m_flag,q_flag,s_flag,obs_time
0,AE000041196,2022-01-01,TAVG,204,H,NaN,S,NaN
1,AEM00041194,2022-01-01,TAVG,211,H,NaN,S,NaN
2,AEM00041217,2022-01-01,TAVG,209,H,NaN,S,NaN
3,AEM00041218,2022-01-01,TAVG,207,H,NaN,S,NaN
4,AG000060390,2022-01-01,TAVG,121,H,NaN,S,NaN
...,...,...,...,...,...,...,...,...
21919285,VQ1VISC0029,2022-09-07,PRCP,221,NaN,NaN,N,NaN
21919286,VQ1VISJ0004,2022-09-07,PRCP,112,NaN,NaN,N,NaN
21919287,VQ1VISJ0005,2022-09-07,PRCP,119,NaN,NaN,N,NaN
21919288,VQ1VIST0003,2022-09-07,PRCP,61,NaN,NaN,N,NaN
